# 11. Brief Tour of the Standard Library - Part II

The following was inspired by: https://docs.python.org/3/tutorial/stdlib2.html

This second tour covers more advanced modules that support professional programming needs. These modules rarely occur in small scripts.

## 11.1. Output Formatting

The [reprlib](https://docs.python.org/3/library/reprlib.html#module-reprlib) module provides a version of [repr()](https://docs.python.org/3/library/functions.html#repr) customized for abbreviated displays of large or deeply nested containers:

In [1]:
import reprlib
reprlib.repr(set('supercalifragilisticexpialidocious'))

"{'a', 'c', 'd', 'e', 'f', 'g', ...}"

The [pprint](https://docs.python.org/3/library/pprint.html#module-pprint) module offers more sophisticated control over printing both built-in and user defined objects in a way that is readable by the interpreter. When the result is longer than one line, the "pretty printer" adds line breaks and indentation to more clearly reveal data structure:

In [2]:
import pprint
t = [[[['black', 'cyan'], 'white', ['green', 'red']], [['magenta',
    'yellow'], 'blue']]]

pprint.pprint(t, width=30)

[[[['black', 'cyan'],
   'white',
   ['green', 'red']],
  [['magenta', 'yellow'],
   'blue']]]


The [textwrap](https://docs.python.org/3/library/textwrap.html#module-textwrap) module formats paragraphs of text to fit a given screen width:

In [3]:
import textwrap
doc = """The wrap() method is just like fill() except that it returns
a list of strings instead of one big string with newlines to separate
the wrapped lines."""

print(textwrap.fill(doc, width=40))

The wrap() method is just like fill()
except that it returns a list of strings
instead of one big string with newlines
to separate the wrapped lines.


The [locale](https://docs.python.org/3/library/locale.html#module-locale) module accesses a database of culture specific data formats. The grouping attribute of locale's format function provides a direct way of formatting numbers with group separators:

In [4]:
import locale
locale.setlocale(locale.LC_ALL, 'English_United States.1252')

Error: unsupported locale setting

In [5]:
conv = locale.localeconv() # get a mapping of conventions

In [6]:
x = 1234567.8
locale.format("%d", x, grouping=True)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: This method will be removed in a future version of Python. Use 'locale.format_string()' instead.
  


'1234567'

In [7]:
locale.format_string("%s%.*f", (conv['currency_symbol'], conv['frac_digits'], x), grouping=True)

'1234567.8000000000465661287307739257812500000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'

## 11.2. Templating

The [string](https://docs.python.org/3/library/string.html#module-string) module includes a versatile [Template](https://docs.python.org/3/library/string.html#string.Template) class with a simplified syntax suitable for editing by end-users. This allows users to customize their applications without having to alter the application.

The format uses placeholder names formed by `$` with valid Python identifiers (alphanumeric characters and underscores). Surrounding the placeholder with braces allows it to be followed by more alphanumeric letters with no intervening spaces. Writing `$$` creates a single escaped `$`:

In [8]:
from string import Template
t = Template('${village}folk send $$10 to $cause.')
t.substitute(village='Nottingham', cause='the ditch fund')

'Nottinghamfolk send $10 to the ditch fund.'

The [substitute()](https://docs.python.org/3/library/string.html#string.Template.substitute) method raises a [KeyError](https://docs.python.org/3/library/exceptions.html#KeyError) when a placeholder is not supplied in a dictionary or a keyword argument. For mail-merge style applications, user supplied data may be incomplete and the [safe_substitute()](https://docs.python.org/3/library/string.html#string.Template.safe_substitute) method may be more appropriate - it will leave placeholders unchanged if data is missing:

In [9]:
t = Template('Return the $item to $owner.')
d = dict(item='unladen swallow')
t.substitute(d)

KeyError: 'owner'

In [10]:
t.safe_substitute(d)

'Return the unladen swallow to $owner.'

Template subclasses can specify a custom delimiter. For example, a batch renaming utility for a photo browser may elect to use percent signs for placeholders such as the current date, image sequence number, or file format:

**NOTE:** Try using `Image_%n%f` to test the following code

In [11]:
import time, os.path
photofiles = ['img_1074.jpg', 'img_1076.jpg', 'img_1077.jpg']
class BatchRename(Template):
    delimiter = '%'
fmt = input('Enter rename style (%d-date %n-seqnum %f-format):  ')

Enter rename style (%d-date %n-seqnum %f-format):  Ashley_%n%f


In [12]:
t = BatchRename(fmt)
date = time.strftime('%d%b%y')
for i, filename in enumerate(photofiles):
    base, ext = os.path.splitext(filename)
    newname = t.substitute(d=date, n=i, f=ext)
    print('{0} --> {1}'.format(filename, newname))

img_1074.jpg --> Ashley_0.jpg
img_1076.jpg --> Ashley_1.jpg
img_1077.jpg --> Ashley_2.jpg


Another application for templating is separating program logic from the details of multiple output formats. This makes it possible to substitute custom templates for XML files, plain text reports, and HTML web reports.

## 11.3. Working with Binary Data Record Layouts

The [struct](https://docs.python.org/3/library/struct.html#module-struct) module provides [pack()](https://docs.python.org/3/library/struct.html#struct.pack) and [unpack()](https://docs.python.org/3/library/struct.html#struct.unpack) functions for working with variable length binary record formats. The following example shows how to loop through header information in a ZIP file without using the [zipfile](https://docs.python.org/3/library/zipfile.html#module-zipfile) module. Pack codes `"H"` and `"I"` represent two and four byte unsigned numbers respectively. The `"<"` indicates that they are standard size and in little-endian byte order:

In [13]:
import struct

with open('myfile.zip', 'rb') as f:
    data = f.read()

start = 0
for i in range(3): # show the first 3 file headers
    start += 14
    fields = struct.unpack('<IIIHH', data[start:start+16])
    crc32, comp_size, uncomp_size, filenamesize, extra_size = fields

    start += 16
    filename = data[start:start+filenamesize]
    start += filenamesize
    extra = data[start:start+extra_size]
    print(filename, hex(crc32), comp_size, uncomp_size)

    start += extra_size + comp_size # skip to the next header

FileNotFoundError: [Errno 2] No such file or directory: 'myfile.zip'

## 11.4. Multi-threading

Threading is a technique for decoupling tasks which are not sequentially dependent. Threads can be used to improve the responsiveness of applications that accept user input while other tasks run in the background. A related use case is running I/O in parallel with computations in another thread.

The following code shows how the high level [threading](https://docs.python.org/3/library/threading.html#module-threading) module can run tasks in background while the main program continues to run:

In [14]:
import threading, zipfile

class AsyncZip(threading.Thread):
    def __init__(self, infile, outfile):
        threading.Thread.__init__(self)
        self.infile = infile
        self.outfile = outfile

    def run(self):
        f = zipfile.ZipFile(self.outfile, 'w', zipfile.ZIP_DEFLATED)
        f.write(self.infile)
        f.close()
        print('Finished background zip of:', self.infile)

background = AsyncZip('mydata.txt', 'myarchive.zip')
background.start()
print('The main program continues to run in foreground.')

background.join() # Wait for the background task to finish
print('Main program waited until background was done.')

The main program continues to run in foreground.
Main program waited until background was done.


Exception in thread Thread-4:
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "<ipython-input-14-23806271f809>", line 11, in run
    f.write(self.infile)
  File "/opt/anaconda3/lib/python3.7/zipfile.py", line 1713, in write
    zinfo = ZipInfo.from_file(filename, arcname)
  File "/opt/anaconda3/lib/python3.7/zipfile.py", line 506, in from_file
    st = os.stat(filename)
FileNotFoundError: [Errno 2] No such file or directory: 'mydata.txt'



The principal challenge of multi-threaded applications is coordinating threads that share data or other resources. To that end, the threading module provides a number of synchronization primitives including locks, events, condition variables, and semaphores.

While those tools are powerful, minor design errors can result in problems that are difficult to reproduce. So, the preferred approach to task coordination is to concentrate all access to a resource in a single thread and then use the [queue](https://docs.python.org/3/library/queue.html#module-queue) module to feed that thread with requests from other threads. Applications using [Queue](https://docs.python.org/3/library/queue.html#queue.Queue) objects for inter-thread communication and coordination are easier to design, more readable, and more reliable.

## 11.5. Logging